In [2]:
from keras.layers import Dense, Dropout, LSTM, Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

C:\Users\konya\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import pandas as pd
import numpy as np

In [30]:
train_data = pd.read_csv("C:\\Users\\konya\\Desktop\\DNA TFBP\\DNA_TranscriptionFactorBindingPrediction\\data\\train.csv");
test_data = pd.read_csv("C:\\Users\\konya\\Desktop\\DNA TFBP\\DNA_TranscriptionFactorBindingPrediction\\data\\test.csv");
train_len = int(len(train_data))
print(train_len)

2000


In [110]:
#split train data into train and validation data
split_ratio = 0.8

train_size = int( train_len * (split_ratio))

#before taking partitions shuffle rows
#this is because all same class tuples are grouped together
train_data = train_data.sample(frac=1).reset_index(drop=False)

X_train = np.array(train_data['sequence'][0:])
Y_train = np.array(train_data['label'][0:])
X_test = np.array(train_data['sequence'][train_size:])
Y_test = np.array(train_data['label'][train_size:])


In [118]:
#preprocessing of DNA sequences
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(split='', char_level=True)
tokenizer.fit_on_texts(X_train)

print(tokenizer.word_counts)
print(tokenizer.document_count)
print(tokenizer.word_index)
print(tokenizer.word_docs)

encoded_X_train = tokenizer.texts_to_sequences(X_train) #mode for text_to_matrix = freq,count,binary,tfidf
print(encoded_X_train[0])

OrderedDict([('C', 6904), ('T', 5713), ('G', 9361), ('A', 6022)])
2000
{'G': 1, 'C': 2, 'A': 3, 'T': 4}
{'T': 1710, 'A': 1787, 'G': 1905, 'C': 1899}
[2, 2, 4, 1, 2, 4, 1, 2, 3, 2, 4, 1, 2, 4]


In [142]:
#LSTM model in keras
def model_init(input_length):
    LSTM_model = Sequential()
    print(input_length)
    LSTM_model.add(Embedding(5,100, input_length = 14))
    
    LSTM_model.add(LSTM(activation="sigmoid", recurrent_activation="hard_sigmoid", return_sequences=True, units=256))
    LSTM_model.add(LSTM(activation="sigmoid", recurrent_activation="hard_sigmoid", units=256))
    LSTM_model.add(Dense(1, activation='sigmoid'))


    LSTM_model.compile(loss='binary_crossentropy', optimizer='adagrad', metrics=['accuracy'])
    return LSTM_model

In [124]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(X_test.shape)

(2000,)
(2000,)
(400,)
(400,)


In [143]:
model = model_init(train_len)

trained_model = model.fit(np.array(encoded_X_train), np.array(Y_train), batch_size=10, epochs=15, validation_split = 0.2, verbose = 1)


2000
Train on 1600 samples, validate on 400 samples
Epoch 1/15
1600/1600 [==============================] - 25s 16ms/step - loss: 0.5691 - acc: 0.7175 - val_loss: 0.3735 - val_acc: 0.8550
Epoch 2/15
1600/1600 [==============================] - 20s 12ms/step - loss: 0.3494 - acc: 0.8512 - val_loss: 0.3896 - val_acc: 0.8150
Epoch 3/15
1600/1600 [==============================] - 20s 12ms/step - loss: 0.3320 - acc: 0.8575 - val_loss: 0.3432 - val_acc: 0.8625
Epoch 4/15
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3305 - acc: 0.8575 - val_loss: 0.3471 - val_acc: 0.8500
Epoch 5/15
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3327 - acc: 0.8531 - val_loss: 0.3380 - val_acc: 0.8625
Epoch 6/15
1600/1600 [==============================] - 20s 13ms/step - loss: 0.3245 - acc: 0.8637 - val_loss: 0.3661 - val_acc: 0.8350
Epoch 7/15
1600/1600 [==============================] - 20s 12ms/step - loss: 0.3237 - acc: 0.8556 - val_loss: 0.3509 - val_acc: 0.8

In [144]:
encoded_X_test = tokenizer.texts_to_sequences(X_test)
score, accuracy = model.evaluate(np.array(encoded_X_test), np.array(Y_test), batch_size=1)
print('Score:', score)
print('Accuracy:', accuracy)

400/400 [==============================] - 3s 7ms/step
Score: 0.34033848020946605
Accuracy: 0.86


In [145]:
sub_X_test =  np.array(test_data['sequence'][0:])
encoded_sub_X_test = tokenizer.texts_to_sequences(sub_X_test)
#preds = model.predict(np.array(encoded_sub_X_test))
#print(preds[:10])
preds = model.predict_classes(np.array(encoded_sub_X_test))
#print(preds[:10])